In [1]:
from gensim.models.doc2vec import *

In [2]:
model = Doc2Vec.load('../store/doc2vec_model_800_pico_19k.mod')

In [5]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
cohen_vec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p'))

In [6]:
X = cohen_vec
X = map(lambda s : [cohen_vec.idx2word[t] for t in s if t > 0], X)

In [7]:
embedds = []
for sent in X :
    embedds.append(model.infer_vector(sent))

In [8]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(1767, 800)

In [9]:
import pandas as pd
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [10]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [12]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.7389371083
{'NSAIDS_processed': 0.7872756659133453, 'ACEInhibitors_processed': 0.73844739355009292, 'SkeletalMuscleRelaxants_processed': 0.6415605404928022, 'Triptans_processed': 0.80936259771742181, 'OralHypoglycemics_processed': 0.77099336852077427, 'CalciumChannelBlockers_processed': 0.56199492059349709, 'BetaBlockers_processed': 0.66646017698021975, 'Estrogens_processed': 0.82345415925297594, 'ADHD_processed': 0.79772319503872524, 'Statins_processed': 0.6920554701017203, 'Antihistamines_processed': 0.68134475826551733, 'ProtonPumpInhibitors_processed': 0.76259864910728814, 'Opiods_processed': 0.74809676303190364, 'AtypicalAntipsychotics_processed': 0.82449444638009972}


In [1]:
dic = eval("{'NSAIDS_processed': 0.7872756659133453, 'ACEInhibitors_processed': 0.73844739355009292, 'SkeletalMuscleRelaxants_processed': 0.6415605404928022, 'Triptans_processed': 0.80936259771742181, 'OralHypoglycemics_processed': 0.77099336852077427, 'CalciumChannelBlockers_processed': 0.56199492059349709, 'BetaBlockers_processed': 0.66646017698021975, 'Estrogens_processed': 0.82345415925297594, 'ADHD_processed': 0.79772319503872524, 'Statins_processed': 0.6920554701017203, 'Antihistamines_processed': 0.68134475826551733, 'ProtonPumpInhibitors_processed': 0.76259864910728814, 'Opiods_processed': 0.74809676303190364, 'AtypicalAntipsychotics_processed': 0.82449444638009972}")
import pandas as pd
pd.Series(dic)

ACEInhibitors_processed              0.738447
ADHD_processed                       0.797723
Antihistamines_processed             0.681345
AtypicalAntipsychotics_processed     0.824494
BetaBlockers_processed               0.666460
CalciumChannelBlockers_processed     0.561995
Estrogens_processed                  0.823454
NSAIDS_processed                     0.787276
Opiods_processed                     0.748097
OralHypoglycemics_processed          0.770993
ProtonPumpInhibitors_processed       0.762599
SkeletalMuscleRelaxants_processed    0.641561
Statins_processed                    0.692055
Triptans_processed                   0.809363
dtype: float64

In [11]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.75958038059
{'NSAIDS_processed': 0.7705994362013846, 'ACEInhibitors_processed': 0.73610855860322055, 'SkeletalMuscleRelaxants_processed': 0.6725699902852601, 'Triptans_processed': 0.82250326078243707, 'OralHypoglycemics_processed': 0.7994939968679311, 'CalciumChannelBlockers_processed': 0.59676833957431563, 'BetaBlockers_processed': 0.64797593016393429, 'Estrogens_processed': 0.84879385900095961, 'ADHD_processed': 0.8239528068801184, 'Statins_processed': 0.75674381637338306, 'Antihistamines_processed': 0.72972267978495309, 'ProtonPumpInhibitors_processed': 0.79289752229497823, 'Opiods_processed': 0.7482198087759685, 'AtypicalAntipsychotics_processed': 0.85134268180028361}
